In [ ]:
from datascience import *
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from IPython.display import Image, display
import ipywidgets as widgets
from scipy import stats
%matplotlib inline

# Tables 

For a collection of things in the world, an array is useful for describing a single attribute of each thing. For example, among the collection of US States, an array could describe the land area of each. Tables extend this idea by describing multiple attributes for each element of a collection.

In most data science applications, we have data about many entities, but we also have several kinds of data about each entity.

When we import data later in this lab, it will import into a table format.

### Analyzing datasets
With just a few table methods, we can answer some interesting questions about datasets.

We can extract single columns, which are arrays themselves, and do math on them (averaging, max, min, etc), which we'll do on real data soon. We can also rearrange the order of rows in a table by the values in any column, add more rows or columns, filter tables to select only rows that meet certain criteria, and much more!

### Tables Essentials!

For your reference, here's a table of all the functions and methods we saw in this lab.

|Name|Example|Purpose|
|-|-|-|
|`Table`|`Table()`|Create an empty table, usually to extend with data|
|`Table.read_table`|`Table.read_table("my_data.csv")`|Create a table from a data file|
|`with_columns`|`tbl = Table().with_columns("N", np.arange(5), "2*N", np.arange(0, 10, 2))`|Create a copy of a table with more columns|
|`column`|`tbl.column("N")`|Create an array containing the elements of a column|
|`sort`|`tbl.sort("N")`|Create a copy of a table sorted by the values in a column|
|`where`|`tbl.where("N", are.above(2))`|Create a copy of a table with only the rows that match some *predicate*|
|`num_rows`|`tbl.num_rows`|Compute the number of rows in a table|
|`num_columns`|`tbl.num_columns`|Compute the number of columns in a table|
|`select`|`tbl.select("N")`|Create a copy of a table with only some of the columns|
|`drop`|`tbl.drop("2*N")`|Create a copy of a table without some of the columns|
|`take`|`tbl.take(np.arange(0, 6, 2))`|Create a copy of the table with only the rows whose indices are in the given array|


---

# UC Berkeley

We're going to start right here at UCB! These data are from Fall 2015.

*Source: UC Corporate Personnel System*

**Note**: STEM includes engineering and computer science, life sciences, math, medicine, other health sciences and physical sciences.

Let's read in a CSV file. A CSV file is a common storage device for spreadsheet data and is also easily manipulated and exported by using programs like Excel.

These data will give us the ratio of female ladder rank equivalent (LRE), which are tenure and tenure track faculty at Berkeley, in the respective divisions.

In [ ]:
UCB_LRE_female = Table.read_table('data/UCB-percent-female-LRE.csv').drop(1)
UCB_LRE_female.show()

We can quickly plot these data on a bar graph, so that we can best visually compare between disciplines over time.

In [ ]:
UCB_LRE_female.barh('Discipline')

What do you notice about the proportions of female LRE faculty across disciplines? Discuss with the people around you.

---

We see in other published materials that over the ten year period there has been an increase in underrepresented minorities at UCB:

<img src="data/gender_subject.png" alt="gender_subject" style="width: 400px;"/>

The increase in the share of ladder-rank and equivalent (LRE) faculty who are underrepresented minorities has largely been due to an increase in the Hispanic/Latino(a) group. Representation by American Indian and African American faculty remains a challenge.

Female LRE faculty have grown in share over time, fueled by increased diversity in hiring. Their proportion differs significantly depending on discipline.

<img src="data/subject_line_graph.png" alt="subject_line_graph" style="width: 400px;"/>

---

## UCOP Payroll Dataset

Let's look at another dataset that has the payroll for all UC empoloyees:

In [ ]:
UCB_data = Table.read_table('data/UCOP.csv')
UCB_data.show(5)

Let's look only at professors:

In [ ]:
rd = UCB_data.select(2,3,4,5).sort(3, descending=True)
professors = rd.where("Title", are.equal_to("PROF-AY"))
professors.show(5)

Big money!

We can visualize the distribution of pay with a histogram, but the histogram (counting frequencies of a specific pay level) will change depending upon the "bin size" of these pay levels. We can make an interactive slider to see this:

In [ ]:
def hist_bins(bin_size=1):
    professors.select(3).hist(bins=np.arange(0,500000,bin_size*2000))

slider = widgets.IntSlider(min=1,max=10,step=1,value=5)
display(widgets.interactive(hist_bins, bin_size=slider))

What are the drawbacks and advantages of different bin sizes?

> Type your response here.

### Salary for males vs. females on the UC payroll

While we don't have gender data in this dataset, we can use a pre-trained machine learning model to predict gender based on first name (we will forget for a moment that creating binary categories of male and female is problematic to begin with). While this is ***certainly not 100% accurate***, it is more like around 80%, we can use it to get a better idea of salaries for different genders.

In [ ]:
from scripts.gender import classify_gender

classify_gender("Daniel"), classify_gender("Katherine")

In [ ]:
professors.append_column("Gender", [classify_gender(name) for name in professors['First Name']])
professors.show(5)

In [ ]:
professors.to_df().groupby(['Gender'])['Gross Pay'].mean()

In [ ]:
professors.to_df().groupby(['Gender'])['Gross Pay'].mean().plot.bar()

What can we conclude from this analysis?

---

## Silicon Valley

These data are compiled from EEO-1 reports from Apple, Twitter, Salesforce, Facebook, Microsoft, and Intel. The EEO-1 is a document required by the federal government that provides the raw numbers of employees in each of the categories below. We summed the most recent data (all from 2014-16) for these companies to get the table below.

In [ ]:
tech_data = Table.read_table('data/eeo-aggregate.csv')
tech_data.show()

We can look at a basic bar chart of all males and females by job category:

In [ ]:
tech_data.select(['Job Categories', 'All Male', 'All Female']).barh('Job Categories')

We can also break down each gender by race:

In [ ]:
tech_data.select(['Job Categories'] + females).barh('Job Categories')

In [ ]:
tech_data.select(['Job Categories'] + males).barh('Job Categories')

What do you see in this data?

---

## Bay Area Census data

Let's read in a CSV file with Bay Area data:

In [ ]:
bay_area = Table().read_table('data/bay_area_data.csv')
bay_area.show(5)

### Job code subset

As you can see above, this table has a lot of information. The variables are in the columns and each row is an observation. First, we will subset this table to only include the occupations we want to analyze. Job codes are listed in the column `OCC2010`. We're going to focus on management and stem jobs.

In [ ]:
job_codes = [10, 20, 30, 100, 110, 120, 130, 140, 150, 160, 220, 300, 310, 330, 350, 360, 410, 420,
             620, 700, 710, 720, 730, 800, 820, 940, 950, 1000, 1010, 1020, 1050, 1060, 1100, 1200, 1220,
             1230, 1240, 1350, 1360, 1400, 1410, 1420, 1430, 1450, 1460, 1540, 1550, 1720, 1910, 1920,
             1980, 2840, 2900, 4000, 4010, 4030, 4050, 4060, 4110, 4120, 4130, 4140, 4150, 4200, 4210,
             4220, 4230, 4250, 4720, 5000, 7720, 7730, 7900, 8000, 8010, 8030, 8060, 8800, 8830, 7700,
             9620, 9630, 9640]

df = bay_area.to_df()
bay_area_cut = Table.from_df(df.loc[df['OCC2010'].isin(job_codes)])
bay_area_cut.show(5)

Although still large, a table with 13110 rows has now decreased to 2550 by selecting rows that match our job_codes array. Let's subset this further by picking out specific variables we want to look at:

In [ ]:
cut_bay_area= bay_area_cut.drop("CPSID","ASECFLAG","HWTSUPP", "HFLAG", "MONTH", "PERNUM", "CPSIDP","WTSUPP")
cut_bay_area.show(5)

The column of job codes in "OCC2010" still does not paint a picture of who is doing which jobs. To solve this, we may add a job sector classification. The array "sector" is created below by the function "job categories". Following the code below, if an array (such as the job code column) is ran through the job_categories function, an array of corresponding sectors is outputted. 

In [ ]:
job_categories = {"STEM": [700, 1000, 1010, 1020, 1050, 1220, 1230, 1240, 1350, 1360, 1400, 1410, 1420, 1430, 1450,1460, 1540, 1550, 1720, 1910, 1920, 1980,2840, 2900,7720, 7730, 7900, 8000, 8010,8030, 8060, 8800, 8830],
                  "SERVICE": [7700, 9620, 9630, 9640, 4000, 4010, 4030, 4050, 4060, 4110, 4120, 4130, 4140, 4150, 4720],
                  "FINANCIAL": [120, 800, 820, 940, 950],
                  "CUSTODIAL": [4200, 4210, 4220, 4230, 4250],
                  "MANAGEMENT": [130, 150, 160, 220, 30, 100, 410, 420],
                  "STEM_MANAGER": [140,300,330, 350, 360, 1060, 1100],
                  "ADMINISTRATOR": [10,20]}

job_categories = dict((v,k) for k in job_categories for v in job_categories[k])

sectors = []
for job in cut_bay_area.column("OCC2010"):
    try:
        sectors.append(job_categories[job])
    except:
        sectors.append("UNKNOWN")
len(sectors)

Now we can add the sector of each individual as a column by using the `with_column` function as seen below. 

In [ ]:
with_sector = cut_bay_area.with_column('SECTOR', sectors)
with_sector

You might have noticed this earlier but race in this table is listed as a number. To make analyis more intuitive, let's change the race codes into what they mean.

In [ ]:
race_dict = {'White': list(range(100,200)),
             'Black': list(range(200,300)),
             'Indigenous': list(range(300,400)),
             'Asian': list(range(400,500)),
             'Pacific Islander': list(range(500,600)),
             'Other': list(range(600,700)),
             'NA': list(range(700,900))}

race_dict = dict((v,k) for k in race_dict for v in race_dict[k])

with_race = Table.from_df(with_sector.to_df().replace({"RACE": race_dict, "SEX": {1: "MALE", 2: "FEMALE"}}))
with_race.show(5)

As you can see, "White" is a pretty big ethnicity group, this may be due to the fact that "White" encompasses a lot according to the 2010 U.S. Census. The definitions of White include Middle Easterners, North Africans and the majority of Hispanic people in the United States. 

In [ ]:
with_race.to_df()['RACE'].value_counts().plot.bar()

In [ ]:
with_race.to_df().groupby(['SECTOR', 'SEX'])['SEX'].count().unstack().plot.bar(stacked=True)

In [ ]:
with_race.to_df().groupby(['SECTOR', 'RACE'])['RACE'].count().unstack().plot.bar(stacked=True)

In [ ]:
for s in set(sectors):
    df = with_race.to_df()
    df[df['SECTOR'] == s].groupby(['RACE', 'SEX'])['SEX'].count().unstack().plot.bar(stacked=True, title=s)

By looking at the average mean of each part of the sample, we see some differences. 

## Income by race and gender

In [ ]:
with_race.to_df().groupby(['SEX'])['INCTOT'].mean()

In [ ]:
with_race.to_df().groupby(['SEX'])['INCTOT'].mean().plot.bar()

In [ ]:
with_race.to_df().groupby(['RACE'])['INCTOT'].mean()

In [ ]:
with_race.to_df().groupby(['RACE'])['INCTOT'].mean().plot.bar()

This type of comparison isn't very reliable. We will perform a p-value test to determine if the change of income across races/ sex is statistically significant. To do this we first need to bootstrap our sample to make a 95% confidence interval of the estimated population mean. 

In [ ]:
def bootstrap_median(original_sample, label, replications):
    '''
    Returns an array of bootstrapped sample medians:
    original_sample: table containing the original sample
    label: label of column containing the variable
    replications: number of bootstrap samples
    '''
    just_one_column = original_sample.select(label)
    medians = make_array()
    for i in np.arange(replications):
        bootstrap_sample = just_one_column.sample()
        resampled_median = percentile(50, bootstrap_sample.column(0))
        medians = np.append(medians, resampled_median)

    return medians

Let's look at `MALE` vs. `FEMALE`:

In [ ]:
median_dict = {}
for i, s in enumerate(['MALE', 'FEMALE']):
    subset = Table.from_df(df[df['SEX'] == s])
    medians= bootstrap_median(subset, "INCTOT", 1000)
    median_dict[s] = medians
    left = percentile(2.5, medians)
    right = percentile(97.5, medians)
    CI = make_array(left, right)
    print("The median 95% Confidence Interval for " + s + " is", CI)
    plt.plot(CI, make_array(i, i), lw=10, label=s)
plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left', ncol=1)
plt.title('Bootstrap Median Income Confidence Intervals')

We can calculate the p-value from the median samples to determine whether the difference is significant:

In [ ]:
stats.ttest_ind(median_dict['MALE'], median_dict['FEMALE'])

We can also look by `RACE`:

In [ ]:
median_dict = {}
for i, r in enumerate(set(df['RACE'])):
    subset = Table.from_df(df[df['RACE'] == r])
    medians= bootstrap_median(subset, "INCTOT", 1000)
    median_dict[r] = medians
    left = percentile(2.5, medians)
    right = percentile(97.5, medians)
    CI = make_array(left, right)
    print("The median 95% Confidence Interval for " + r + " is", CI)
    plt.plot(CI, make_array(i, i), lw=10, label=r)
plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left', ncol=1)
plt.title('Bootstrap Median Income Confidence Intervals')

We can do a one way F test to see if there is significance in the difference between the median samples:

In [ ]:
stats.f_oneway(median_dict['NA'], median_dict['Other'], median_dict['Indigenous'], median_dict['White'], median_dict['Black'])

We can use a fancy tool to give us all the combinations of `SEX` and `RACE` and then get the confidence intervals for those:

In [ ]:
import itertools

combos = [i for i in itertools.product(set(df['RACE']), ['MALE', 'FEMALE'])]
combos

In [ ]:
for i, c in enumerate(combos):
    subset = df[df['RACE'] == c[0]]
    subset = Table.from_df(subset[subset['SEX'] == c[1]])
    medians= bootstrap_median(subset, "INCTOT", 1000)
    left = percentile(2.5, medians)
    right = percentile(97.5, medians)
    CI = make_array(left, right)
    print("The median 95% Confidence Interval for " + c[0] + ' ' + c[1] + " is", CI)
    plt.plot(CI, make_array(i, i), lw=10, label=c[0] + ' ' + c[1])
plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left', ncol=1)
plt.title('Bootstrap Median Income Confidence Intervals')

---

## Compared to entire Bay Area census sample

So how does our tech-biased subset compare to the entire census subset of the Bay Area? First we'll do some quick processing to get out non-responses and relabel the Bay Area subset: 

In [ ]:
bay_area = bay_area.to_df().replace({"RACE": race_dict, "SEX": {1: "MALE", 2: "FEMALE"}})
bay_area = bay_area[bay_area["INCTOT"] != 0]
bay_area = bay_area[bay_area["INCTOT"] != 99999999]

We can then look at income by `SEX` and `RACE`. Let's start with `SEX`.

### `SEX`

In [ ]:
bay_area.groupby(['SEX'])['INCTOT'].mean().plot.bar()

We'll recall that our biased subset had much higher means, but similar disparity:

In [ ]:
with_race.to_df().groupby(['SEX'])['INCTOT'].mean().plot.bar()n

UCB is doing much better than both of these subsets:

In [ ]:
professors.to_df().groupby(['Gender'])['Gross Pay'].mean().plot.bar()

### `RACE`
We can also look at `RACE` in the larger Bay Area subset:

In [ ]:
bay_area.groupby(['RACE'])['INCTOT'].mean()

In [ ]:
bay_area.groupby(['RACE'])['INCTOT'].mean().plot.bar()

Our biased subset was also great here:

In [ ]:
with_race.to_df().groupby(['RACE'])['INCTOT'].mean()

In [ ]:
with_race.to_df().groupby(['RACE'])['INCTOT'].mean().plot.bar()

Where are the disparities the worst? What does this tell us about the tech industry and management?